In [1]:
import torch
import torch.optim as optim
import os
import math

In [2]:
print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}")

MPS 장치를 지원하도록 build가 되었는가? True
MPS 장치가 사용 가능한가? True


In [3]:
os.chdir('/Users/seungyeonlee/Documents/GitHub/24-2-TicTacToe/승연')
from config import *

from enemy_agents import *
from environment import *
from evaluate import *
from file_save_load import *
from mcts import *
from net import *
from train_network import *
from visualizing import *
from connect_CNC import *

In [4]:
env = Environment(STATE_SIZE, WIN_CONDITION)
model = load_model(path=os.getcwd(), file="test_model_best")
agent = Mcts(model, temperature=0)

alphazero_first = input("Is Alpha-Zero first?: (True/False)")
agent_turn = "O" if alphazero_first else "X"

serial = Serial('/dev/cu.usbserial-110', alphazero_first)

Create '/Users/seungyeonlee/Documents/GitHub/24-2-TicTacToe/승연/test_model_best.pkl'
Communication Successfully started


In [5]:
# import
def make_state(raw_state):
    my_actions = np.where(raw_state == 1)[0]
    enemy_actions = np.where(raw_state == -1)[0]
    my_state = np.zeros(STATE_SIZE)
    my_state[my_actions] = 1
    enemy_state = np.zeros(STATE_SIZE)
    enemy_state[enemy_actions] = 1

    state = State(my_state = my_state, enemy_state = enemy_state)

    return state

In [15]:
# Connect robot 
# ser = connect_serial('/dev/cu.usbserial-110')
# waiting_robot(ser)
# draw_game_board(ser)

state = State()
is_done = False

alphazero_first = input("Is Alpha-Zero first?: (True/False)")
agent_turn = "O" if alphazero_first else "X"

if alphazero_first:
    while not is_done:
        # agent turn
        action = agent.get_action(state)
        action_command = f'{action}'
        serial.send_to_robot(agent_turn, '1')
        state, is_done, is_lose = env.step(state, action)

        # human turn
        if not is_done:
            # camera_start(state)
            raw_state = image_to_state(state)
            state = make_state(raw_state)
            is_done = state.check_done()

else:
    while not is_done:
        # human turn
        raw_state = image_to_state(state)
        state = make_state(raw_state)
        is_done = state.check_done()

        # agent turn
        if not is_done:
            action = agent.get_action(state)
            serial.send_to_robot(agent_turn, str(action))
            state, is_done, is_lose = env.step(state, action)


reward = env.get_first_reward(state)
first_player = "Alpha-Zero" if alphazero_first else "You"
second_player = "You" if alphazero_first else "Alpha-Zero"

print(f" << Game Over! {first_player if reward == 1 else second_player} Win!>>")

Sent command: O
Arduino Response: Setting O

Sent command: 1
Arduino Response: Done: drawCircle



NameError: name 'image_to_state' is not defined

In [10]:
serial.waiting_robot()

KeyboardInterrupt: 

In [12]:
serial.send_to_robot('O', '1')

Sent command: O
Arduino Response: Setting O

Sent command: 1
Arduino Response: Done: drawCircle

